In [1]:
import pandas as pd
import geopandas as gpd
# Operations on geometries
import shapely
import numpy as np
import math
import xarray as xr
# Plotting
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
#import wradlib as wrl
import regionmask
from pyproj import Transformer
import pyproj
from pyproj import CRS
import shapely.vectorized as sv

from utils.utils_shapes import *

sns.set()
%matplotlib inline

In [2]:
# Path files
PATH = '/Users/noeliaotero/Documents/OCCR/data/'

In [3]:
# netcdf files 
t2max=xr.open_mfdataset(PATH + '/meteoswiss/TmaxD_v2.0_longlat/*nc', combine='by_coords')
t2abs=xr.open_mfdataset(PATH + '/meteoswiss/TabsD_v2.0_longlat/*nc', combine='by_coords')
tp=xr.open_mfdataset(PATH + '/meteoswiss/RhiresD_v2.0_lonlat/*nc', combine='by_coords')

In [4]:
f_id = '/Output_data/csv/ID_Names.csv'
# read the ID to select only the interesting
ids = pd.read_csv(PATH + f_id)

In [5]:
# open and read each shapefile
shp_Leventina =  gpd.read_file(PATH + 'Shapefiles_powerplants/Leventina/193404.shp')
shp_Blenio =  gpd.read_file(PATH + 'Shapefiles_powerplants/Blenio/100367.shp')
shp_ElectraMassa =  gpd.read_file(PATH + 'Shapefiles_powerplants/Electra-Massa/191746.shp')
shp_Emosson =  gpd.read_file(PATH + 'Shapefiles_powerplants/Emosson/178852.shp')
shp_Mauvoisin =  gpd.read_file(PATH + 'Shapefiles_powerplants/Mauvoisin/104523.shp')
shp_Rheinfelden =  gpd.read_file(PATH + 'Shapefiles_powerplants/Rheinfelden/142378.shp')

In [6]:
all_shp=gpd.GeoDataFrame(pd.concat([shp_Leventina, shp_Blenio, shp_ElectraMassa, shp_Emosson, shp_Mauvoisin, shp_Rheinfelden ]))
all_shp

,id,ezgnr,ar_1903,watr_nm,place,geometry
0,193404,None,23.0,Foss,Quinto,"POLYGON ((8.76996 46.53418, 8.76999 46.53406, ..."
0,100367,None,36.6,Riale di Luzzone,Blenio,"POLYGON ((9.02313 46.53831, 9.02313 46.53831, ..."
0,191746,None,198.3,Massa,Naters,"POLYGON ((8.01645 46.37183, 8.01631 46.37170, ..."
0,178852,None,29.7,Barberine,Finhaut,"POLYGON ((6.89257 46.04732, 6.89257 46.04732, ..."
0,104523,None,114.5,Dranse de Bagnes,Bagnes,"POLYGON ((7.39536 45.90643, 7.39535 45.90627, ..."
0,142378,None,34520.2,Rhein,None,"POLYGON ((8.99981 46.61115, 8.99901 46.61117, ..."


In [7]:
nn = len(all_shp.id)

In [8]:
# print dimensions
print(t2abs.dims)
print(t2max.dims)
print(tp.dims)

Frozen({'time': 15706, 'lon': 241, 'lat': 103})
Frozen({'time': 15706, 'lon': 241, 'lat': 103})
Frozen({'time': 15706, 'lon': 241, 'lat': 121})


In [10]:
nuts_mask_poly = regionmask.Regions(name = 'nuts_mask', numbers = list(range(0,nn)), names = list(all_shp.id), abbrevs = list(all_shp.id), outlines = list(all_shp.geometry.values[i] for i in range(0,nn)))


In [11]:
# Mean temperature
t2m_dat_catch=extract_nuts_TS(t2abs, nuts_mask_poly)

0
1
2
3
4
5


In [12]:
#Maximum temperature
t2max_dat_catch=extract_nuts_TS(t2max, nuts_mask_poly)

0
1
2
3
4
5


In [13]:
# Precipititation
tp_dat_catch=extract_nuts_TS(tp, nuts_mask_poly)

0
1
2
3
4
5


In [15]:
t2m_catch=metaverage_catch(t2m_dat_catch,'TabsD', nuts_mask_poly)
t2max_catch=metaverage_catch(t2max_dat_catch,'TmaxD', nuts_mask_poly)
tp_catch=metaverage_catch(tp_dat_catch,'RhiresD', nuts_mask_poly)

In [20]:
# rename  'AET-Leventina' 'Blenio' 'Massa' 'Emosson' 'Mauvoisin' 'Rheinfelden'
t2m_catch.columns = ['date', '2364', '2494', '2342', '2053', '2117', '2091']
t2max_catch.columns = ['date', '2364', '2494', '2342', '2053', '2117', '2091']
tp_catch.columns = ['date', '2364', '2494', '2342', '2053', '2117', '2091']

In [21]:
dir_out = PATH + "/Output_data/csv/Observed_discharge_entsoe_powerplants/meteovars_avg_finalpowerplants/"
# saving the dataframe 
t2m_catch.to_csv(dir_out + 't2m_197901_202112.csv', index = False, header=True)
t2max_catch.to_csv(dir_out + 't2max_197901_202112.csv', index = False, header=True)
tp_catch.to_csv(dir_out + 'tp_197901_202112.csv', index = False, header=True)